# Analysis: ICE detainers in Travis County, Texas
This notebook has the scripts to clean and analyze data returned in response to a request for ICE detainer records in Travis County. The sheriff posts [PDFs of daily reports](https://www.tcsheriff.org/inmate-jail-info/ice-listing) generated by database software, but the Statesman requested, and received, a spreadsheet version of these reports. This file lives at `raw_data/ICE - Detainer Added.xlsx`.

In [15]:
%%bash

# use csvkit to turn the spreadsheet into a CSV
in2csv "raw_data/ICE - Detainer Added.xlsx" > raw_data/data.csv

# trim six lines from the head and 10 from the tail
count=$(wc -l < raw_data/data.csv | sed 's/ //g'); trim=$(echo $count - 10 | bc); \
tail -n +6 raw_data/data.csv | head -n $trim > raw_data/trimmed_data.csv

# report line count
wc -l < raw_data/trimmed_data.csv | sed 's/ //g'

53439


/Library/Python/2.7/site-packages/openpyxl/workbook/names/named_range.py:121: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


In [52]:
from __future__ import print_function

import csv
from collections import Counter

import agate
from text_transforms import COUNTRIES, VIOLENT_CRIMES, FELONIES


def name_unmangler(name_str):
    try:
        name_split = name_str.split(',')
        return (name_split[0], name_split[1])
    except:
        return name_str


with open('raw_data/trimmed_data.csv', 'r') as infile:
    data = csv.reader(infile, delimiter=',')

    # set initial defaults
    inmate_dict = {}
    new_record = False
    booking_id = None
    
    for row in data:
        if new_record:
            booking_id = row[1].strip()
            name = row[2].strip()
            race = row[3].strip()
            sex = row[4].strip()
            age = row[6].strip()
            booking_date = row[7].strip()
            nativity = row[8].strip()
            nativity_clean = nativity
            for country_set in COUNTRIES:
                if nativity == country_set[0]:
                    nativity_clean = country_set[1]            
            
            inmate_dict[booking_id] = {
                'last_name': name_unmangler(name)[0],
                'rest_name': name_unmangler(name)[1],
                'race': race,
                'sex': sex,
                'age': age,
                'booking_date': booking_date,
                'nativity': nativity,
                'nativity_clean': nativity_clean,
                'charges': []
            }
            
            new_record = False
        else:
            rec = inmate_dict.get(booking_id, None)
            if rec:
                if ''.join(row).strip() != '' and row[1].strip() != 'Charge':
                    charge = {}
                    charge['charge_id'] = row[1].strip()
                    charge['charge_description'] = row[2].strip()
                    charge['charge_level'] = row[3].strip()
                    charge['sentence'] = row[4].strip()
                    charge['disposition_date'] = row[6].strip()
                    charge['disposition_description'] = row[8].strip()
                    rec['charges'].append(charge)
        
        # new records are delineated by a blank row
        if ''.join(row).strip() == '' or row[1] == 'Booking No':
            new_record = True
            
    print('who has two thumbs and a dict with', len(inmate_dict), 'keys')
    print('you do that\'s who')

who has two thumbs and a dict with 9533 keys
you do that's who


### Agate it up for basic analysis

In [57]:
# set up some basic analysis for agate
rows = []
column_names = ['booking_id', 'booking_date', 'booking_month_year', 'nativity', 'sex',
                'age', 'race', 'has_felony', 'has_violent']
column_types = [agate.Text(), agate.Date(), agate.Text(), agate.Text(), agate.Text(), 
                agate.Number(), agate.Text(), agate.Boolean(), agate.Boolean()]

# and a list for a counter to get a basic frequency count of charges
charge_list = []

for key in inmate_dict:
    ls = []
    has_felony = False
    has_violent = False
    
    month_year = inmate_dict[key]['booking_date'][:7]

    ls.append(key)
    ls.append(inmate_dict[key]['booking_date'])
    ls.append(month_year)
    ls.append(inmate_dict[key]['nativity_clean'])
    ls.append(inmate_dict[key]['sex'])
    ls.append(inmate_dict[key]['age'])
    ls.append(inmate_dict[key]['race'])
    
    for charge in inmate_dict[key]['charges']:
        charge_list.append(charge['charge_description'])

        if charge['charge_level'] in FELONIES:
            has_felony = True
        if charge['charge_description'].upper() in VIOLENT_CRIMES:
            has_violent = True
            
    ls.append(has_felony)
    ls.append(has_violent)

    rows.append(ls)

print("10 MOST COMMON CHARGES\n----------------------")
for charge in Counter(charge_list).most_common(10):
    print(charge[0] + ":", charge[1])

print('\n')
    
table = agate.Table(rows, column_names, column_types).order_by('booking_date', reverse=True)

# grab some overall grouped totals

# by country
by_country = table.group_by('nativity')

country_totals = by_country.aggregate([
    ('count', agate.Count())
])

sorted_country_totals = country_totals.order_by('count', reverse=True)

print("BY REPORTED COUNTRY OF ORIGIN")
sorted_country_totals.print_table(max_rows=10)
print('\n')

# by sex
by_sex = table.group_by('sex')

sex_totals = by_sex.aggregate([
    ('count', agate.Count())
])

sorted_sex_totals = sex_totals.order_by('count', reverse=True)

print("BY SEX")
sorted_sex_totals.print_table()
print('\n')

# by race
by_race = table.group_by('race')

race_totals = by_race.aggregate([
    ('count', agate.Count())
])

sorted_race_totals = race_totals.order_by('count', reverse=True)

print("BY RACE")
sorted_race_totals.print_table(max_rows=10)
print('\n')

# by age
print("BY AGE")
binned_ages = table.bins('age', 10, 0, 100)
binned_ages.print_bars('age', 'Count', width=80)
print('\n')

# by felony status
by_felony = table.group_by('has_felony')

print("BY FELONY STATUS")
print(by_felony)
print('\n')

# by violent crime status
by_violent = table.group_by('has_violent')

print("BY VIOLENT CRIME STATUS")
print(by_violent)

10 MOST COMMON CHARGES
----------------------
ICE DETAINER: 9602
DRIVING WHILE INTOXICATED: 2849
ASSLT CAUSES BODILY INJ:FAMILY MEMBER (MA): 1164
TRAFFIC OFFENSE MULTIPLE: 1077
PUBLIC INTOXICATION: 766
DRIVING WHILE INTOXICATED 2ND: 514
DRIVING WHILE INTOXICATED BAC>=0.15: 501
FAIL TO ID GIVING FALSE/FICTIOUS INFO (MB): 424
POSS MARIJ <2OZ (MB): 404
FAIL TO ID FUGITIVE INTENT GIVE FALSE (MA): 382


BY REPORTED COUNTRY OF ORIGIN
|-----------------+--------|
|  nativity       | count  |
|-----------------+--------|
|  Mexico         | 7,555  |
|  Honduras       |   771  |
|  Guatemala      |   361  |
|  United States  |   269  |
|  El Salvador    |   228  |
|  Cuba           |    43  |
|  Nicaragua      |    42  |
|  Vanuatu        |    16  |
|  Vietnam        |    14  |
|  United Kingdom |    12  |
|  ...            |   ...  |
|-----------------+--------|


BY SEX
|------+--------|
|  sex | count  |
|------+--------|
|  M   | 9,041  |
|  F   |   492  |
|------+--------|


BY RACE
|-----

### Breakdown by month

In [48]:
table.print_table()

grouped_by_month = table.group_by('booking_month_year')

for grouped_table in grouped_by_month:
    print(grouped_table.select('booking_month_year').rows[0].values()[0])
    grouped_by_sex = grouped_table.group_by('sex')
    sex_totals = grouped_by_sex.aggregate([
        ('count', agate.Count())
    ])

    sorted_sex_totals = sex_totals.order_by('count', reverse=True)

    print("BY SEX")
    sorted_sex_totals.print_table()
    print('\n')

|-------------+--------------+--------------------+-----------+-----+-----+------|
|  booking_id | booking_date | booking_month_year | nativity  | sex | age | ...  |
|-------------+--------------+--------------------+-----------+-----+-----+------|
|  1636108    |   2016-10-03 | 2016-10            | Mexico    | M   | 42  | ...  |
|  1636080    |   2016-10-02 | 2016-10            | Nicaragua | M   | 30  | ...  |
|  1635995    |   2016-10-02 | 2016-10            | Mexico    | M   | 41  | ...  |
|  1636067    |   2016-10-02 | 2016-10            | Honduras  | M   | 42  | ...  |
|  1636052    |   2016-10-02 | 2016-10            | Mexico    | M   | 26  | ...  |
|  1635891    |   2016-10-01 | 2016-10            | Mexico    | M   | 38  | ...  |
|  1635767    |   2016-09-30 | 2016-09            | Honduras  | M   | 36  | ...  |
|  1635848    |   2016-09-30 | 2016-09            | Mexico    | M   | 30  | ...  |
|  1635789    |   2016-09-30 | 2016-09            | Honduras  | M   | 31  | ...  |
|  1

### Questions to explore
- breakdown of detainers by month, then by race, sex, nativity, age, charge, charge level, etc.
- for each month, how many of total detainees had felonies?
- did ending secure communities have an effect on # or type of detainees?